In [1]:
import pandas as pd
import numpy as np
from itertools import *
pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', None)

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

In [5]:
from statistic import Statistic
from utils import UtilsKy
from factors import Factor
from analyzer import HelperAnalyzer, AnalyzerPrediction

In [6]:
from naive_bayes import Bayes

In [7]:
# kyw3
path_data = '/mnt/files/workdata/work/merchants/merchant_33_kyw3_2020-06-05/04_experiments/ex_01_some_teach/'
db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype=str, encoding='cp1251')
db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str, encoding='cp1251')
white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

In [9]:
str(list(db_teach))

"['amount', 'amount_deviation', 'bank_currency', 'bin', 'city', 'count_months_to_end_card', 'day_of_week', 'gender2', 'hour', 'id', 'is_city_resolved', 'is_gender_undefined', 'latitude', 'longitude', 'order_id', 'phone_2_norm', 'status']"

In [10]:
path_resolving_file = '/mnt/files/workdata/work/merchants/merchant_33_kyw3_2020-06-05/02_pure_data/kyw3_for_resolve.csv'
df_resolving = pd.read_csv(path_resolving_file, dtype=str, encoding='cp1251')

In [11]:
str(list(df_resolving))

"['order_id', 'type', 'status', 'date', 'processed', 'amount', 'currency', 'card_holder', 'email', 'phone', 'address', 'city_from_order', 'country', 'zip', 'ip', 'project_id', 'reported', 'status_chargeback', 'date_cb_only', 'card_masked', 'card_expire', 'amount_in_currency', 'date_only', 'id']"

In [12]:
features_res = ['country', 'card_holder', 'city_from_order', 'zip', 'ip', 'card_expire', 'amount_in_currency', 'order_id']

In [13]:
db_teach = db_teach.merge(df_resolving[features_res], left_on='order_id', right_on='order_id', how='left')
db_test = db_test.merge(df_resolving[features_res], left_on='order_id', right_on='order_id', how='left')

In [14]:
str(list(db_test))

"['amount', 'amount_deviation', 'bank_currency', 'bin', 'city', 'count_months_to_end_card', 'day_of_week', 'gender2', 'hour', 'id', 'is_city_resolved', 'is_gender_undefined', 'latitude', 'longitude', 'order_id', 'phone_2_norm', 'status', 'country', 'card_holder', 'city_from_order', 'zip', 'ip', 'card_expire', 'amount_in_currency']"

In [25]:
mean_longitude_usa = -97

In [26]:
db_teach.longitude = np.where(db_teach.longitude == 'undefined', mean_longitude_usa, db_teach.longitude)
db_teach.longitude = pd.to_numeric(db_teach.longitude)
db_teach.longitude = db_teach.longitude.apply(lambda x: round(x))

In [31]:
db_test.longitude = np.where(db_test.longitude == 'undefined', mean_longitude_usa, db_test.longitude)
db_test.longitude = pd.to_numeric(db_test.longitude)
db_test.longitude = db_test.longitude.apply(lambda x: round(x))

In [28]:
mean_latitude_usa = 38

In [29]:
db_teach.latitude = np.where(db_teach.latitude == 'undefined', mean_latitude_usa, db_teach.latitude)
db_teach.latitude = pd.to_numeric(db_teach.latitude)
db_teach.latitude = db_teach.latitude.apply(lambda x: round(x))

In [32]:
db_test.latitude = np.where(db_test.latitude == 'undefined', mean_latitude_usa, db_test.latitude)
db_test.latitude = pd.to_numeric(db_test.latitude)
db_test.latitude = db_test.latitude.apply(lambda x: round(x))

In [40]:
def change_ip(ip):
    x = str.split(ip, '.') 
    x = x[0] + '.' + x[1]
    return x

In [44]:
db_teach.ip = db_teach.ip.apply(lambda x: change_ip(x))
db_test.ip = db_test.ip.apply(lambda x: change_ip(x))

In [50]:
db_teach.amount_in_currency = pd.to_numeric(db_teach.amount_in_currency)
db_teach.amount_in_currency = db_teach.amount_in_currency.apply(lambda x: round(x))

In [51]:
db_test.amount_in_currency = pd.to_numeric(db_test.amount_in_currency)
db_test.amount_in_currency = db_test.amount_in_currency.apply(lambda x: round(x))

In [55]:
COL_FACTORS = ['amount_deviation', 'bank_currency', 'bin', 'city', 'count_months_to_end_card', 'day_of_week', 
               'gender2', 'hour', 'is_city_resolved', 'is_gender_undefined', 'latitude', 'longitude', 
               'phone_2_norm', 'status', 'country', 'card_holder', 'city_from_order', 'zip', 'ip', 
               'card_expire']

In [56]:
main_factors = ['amount_deviation']
extra_factors = ['bank_currency', 'bin', 'city', 'count_months_to_end_card', 'day_of_week', 
               'gender2', 'hour', 'is_city_resolved', 'is_gender_undefined', 'latitude', 'longitude', 
               'phone_2_norm', 'country', 'card_holder', 'city_from_order', 'zip', 'ip', 
               'card_expire']

In [402]:
# COL_FACTORS = COL_FACTORS + main_factors + extra_factors
# COL_FACTORS = list(set(COL_FACTORS))

In [57]:
result_df_amount = None
analyzer_prediction =  AnalyzerPrediction(db_teach, db_test, None)

In [58]:
teach = db_teach[COL_FACTORS].copy()
test = db_test[COL_FACTORS].copy()

In [59]:
model = Bayes() # calculation_type='amount'
model.fit(teach)
model.create_test_weight(test)

factor_list = ['amount_deviation', 'bank_currency', 'bin', 'city', 'count_months_to_end_card', 'day_of_week', 'gender2', 'hour', 'is_city_resolved', 'is_gender_undefined', 'latitude', 'longitude', 'phone_2_norm', 'country', 'card_holder', 'city_from_order', 'zip', 'ip', 'card_expire']


In [60]:
result_df_amount = None
for i in list(range(len(extra_factors)+1)):
    for comb_factors in combinations(extra_factors, i):
        comb_factors = list(comb_factors)
        factor_list = main_factors + comb_factors
        
        test_pred = model.predict_proba(col_names=factor_list)
        db_test["probability"] = test_pred[:, 1]
        description = str(comb_factors)
        result_df_amount = analyzer_prediction.get_table_prediction(description=description, result_df=result_df_amount, metric="amount")


In [61]:
n = result_df_amount.shape[0]
sub_rows = list(range(n))[::2]
stat_best = result_df_amount.copy().iloc[sub_rows,:]

col_names = [col for col in stat_best.columns if col.startswith('p_') ] 
stat_best.loc[:, col_names] = stat_best.loc[:, col_names].astype(float)
stat_best = stat_best.sort_values(by="rating", ascending=False)

In [62]:
stat_best.iloc[:,:11]

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating
209626,"['day_of_week', 'hour', 'is_city_resolved', 'l...",7.96,11.74,12.74,14.09,15.75,16.13,18.27,26.09,39.93,128.30
258246,"['bank_currency', 'day_of_week', 'hour', 'is_c...",7.52,11.39,12.74,14.09,15.88,17.01,18.68,26.09,39.95,127.66
306780,"['day_of_week', 'gender2', 'hour', 'is_city_re...",4.81,8.50,11.29,15.99,17.65,19.39,21.25,28.19,40.78,127.21
120906,"['day_of_week', 'hour', 'is_city_resolved', 'l...",7.86,11.64,12.64,13.99,15.65,16.03,17.81,25.69,39.70,127.17
159802,"['bank_currency', 'day_of_week', 'hour', 'is_c...",7.42,11.29,12.64,13.99,15.78,16.91,18.58,25.99,39.85,126.96
355400,"['bank_currency', 'day_of_week', 'gender2', 'h...",4.81,8.06,11.29,16.01,17.67,19.41,21.27,28.21,40.78,126.83
258106,"['bank_currency', 'day_of_week', 'hour', 'is_c...",7.18,10.36,11.77,13.39,16.04,17.93,18.92,27.32,40.56,126.62
...,...,...,...,...,...,...,...,...,...,...,...
10088,"['bank_currency', 'city', 'country', 'card_hol...",0.36,0.76,2.01,2.65,4.51,5.54,6.07,8.24,15.91,34.44
5108,"['city', 'is_city_resolved', 'card_holder', 'z...",0.36,0.76,2.27,2.82,4.84,5.47,6.07,8.14,15.20,34.39


In [63]:
stat_best.iloc[0,0]

"['day_of_week', 'hour', 'is_city_resolved', 'latitude', 'longitude', 'phone_2_norm', 'country', 'card_expire']"

In [ ]:
def update_bin(bin:str)->str: 
    x = pd.to_numeric(bin[4])
    x = x//2
    return bin[:3] + str(x)    

In [ ]:
q_amount = [0, 0.15, 0.25, 0.50, 0.75, 0.90]
x = 0.25
n = len(q_amount)
for i in range(1, n):
    if x >= q_amount[i - 1] and  x < q_amount[i]:
        print( i)
print(' n={}'.format(n))